In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import BayesianRidge
# supportive models
import xgboost as xgb
# feature selection (from supportive model)
from sklearn.feature_selection import SelectFromModel

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

def get_models():
        return {
            'LinearRegression': linear_model.LinearRegression(),
            'RandomForestRegressor': RandomForestRegressor(n_estimators = 30, random_state = 0),
            # 'MLPRegressor': MLPRegressor(),
            'BayesianRidge': BayesianRidge(),
        }


def train_models(models, x, y):
    for name, i_model in models.items():
        i_model.fit(x, y)
    return models    

def competite_model(models, x, y):
    best_score = -1
    winner = next(iter(models)) # first key
    for name, i_model in models.items():
        if i_model.score(x, y) > best_score:
            winner = name
    return winner       

def create_model():
    # LinearRegression
    # regr = linear_model.LinearRegression()

    # random forest
    # regr = RandomForestRegressor(n_estimators = 30, random_state = 0)

    # neural network
    # regr = MLPRegressor()

    # BayesianRidge
    regr = BayesianRidge()
    return regr

# Any results you write to the current directory are saved as output.

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train_y = train['y']
train = train.drop(['y', 'ID'], 1)

train_x = pd.get_dummies(train)
train_x = train_x.reindex_axis(sorted(train_x.columns), axis=1)

test_x = pd.get_dummies(test)
test_x = test_x.drop('ID', 1)
# remove untrained features
unknonw_columns = test_x.columns.difference(train_x.columns)
test_x = test_x.drop(unknonw_columns.tolist(), 1)
missing_columns = train_x.columns.difference(test_x.columns)
# test_x = pd.concat([test_x,pd.DataFrame(columns=missing_columns)])
for col in missing_columns:
    test_x[col] = 0
test_x = test_x.reindex_axis(sorted(test_x.columns), axis=1)    
test_x.head(3)

In [ ]:
test_x = pd.get_dummies(test)
test_x = test_x.drop('ID', 1)
# remove untrained features
unknonw_columns = test_x.columns.difference(train_x.columns)
test_x = test_x.drop(unknonw_columns.tolist(), 1)
missing_columns = train_x.columns.difference(test_x.columns)
# test_x = pd.concat([test_x,pd.DataFrame(columns=missing_columns)])
for col in missing_columns:
    test_x[col] = 0
test_x = test_x.reindex_axis(sorted(test_x.columns), axis=1)

xgb_params = {
    'n_trees': 500, 
    'eta': 0.004,
    'max_depth': 4,
    'subsample': 0.95,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

clf = xgb.XGBRegressor(**xgb_params)
clf = clf.fit(train_x, train_y)
model = SelectFromModel(clf, prefit=True)

train_x = model.transform(train_x)
test_x = model.transform(test_x)

train_cross_x, train_cross_test_x, train_cross_y, train_cross_test_y = train_test_split(train_x, train_y, test_size=0.01)

pd.DataFrame(train_cross_y).head(4)

In [ ]:
pd.DataFrame(train_x).head(4)

In [ ]:
#MLPRegressor().fit(train_x, train_y)
#linear_model.LinearRegression().fit(train_cross_x, train_cross_y)
#RandomForestRegressor(n_estimators = 30, random_state = 0).fit(train_cross_x, train_cross_y)
#BayesianRidge().fit(train_x, train_y)
# next(train_cross_x.iterrows())[1]
models = train_models(get_models(), train_cross_x, train_cross_y)

In [ ]:
pd.DataFrame(train_cross_x).head(4)

In [ ]:
train_cross_y = pd.DataFrame(train_cross_y.get_values())
train_cross_x = pd.DataFrame(train_cross_x)

In [ ]:
train_cross_models = list()
train_cross_y_values = train_cross_y.get_values()
for idx, x in train_cross_x.iterrows():
    m = competite_model(models, x.get_values().reshape(1, -1), train_cross_y_values[idx])
    train_cross_models.append(m)
train_cross_models_df = pd.DataFrame(train_cross_x)
train_cross_models_df['models'] = train_cross_models

In [ ]:
for m in train_cross_models_df['models']:
    print(m)

In [ ]:
regr = create_model()
regr.fit(train_cross_x, train_cross_y)
regr.score(train_cross_test_x, train_cross_test_y)

In [ ]:
trained_cross_models = train_models(get_models(), train_cross_x, train_cross_y)

out = {}
for name, m in trained_cross_models.items():
    out[name] = m.score(train_cross_test_x, train_cross_test_y)
out    

train_cross_x_df = pd.DataFrame(train_cross_x)
train_cross_y_df = pd.DataFrame({'y': train_cross_y})
train_cross =  pd.concat([train_cross_x_df, train_cross_y_df], axis=1)
print(train_cross.columns.values)
model_series = list()
for idx, row in train_cross.iterrows():
    print(row)
    m = competite_model(trained_cross_models, row.drop('y').reshape(1, -1), 
                        row['y'].reshape(-1, 1))
    print(m)
    model_series.append([m])
model_series    
# linear_model.LinearRegression().fit(train_cross_x, train_cross_y)
# RandomForestRegressor(n_estimators = 30, random_state = 0).fit(train_cross_x, train_cross_y)
# MLPRegressor().fit(train_cross_x, train_cross_y)
# BayesianRidge().fit(train_cross_x, train_cross_y)

In [ ]:
#regr.predict(train_cross_test_x)
#train_cross_test_y.tolist()
plot_offset = 700
pd.DataFrame({'predicted': regr.predict(train_cross_test_x[plot_offset:plot_offset+20]), 
              'expected': train_cross_test_y.tolist()[plot_offset:plot_offset+20]}).plot(kind='bar')

In [ ]:
regr = create_model()

regr.fit(train_x, train_y)
regr.score(train_x, train_y)

In [ ]:
# predict
test_y = regr.predict(test_x)
test_y

# write results
solution = pd.DataFrame({"ID":test.ID, "y":test_y}) 
solution.to_csv('predicted.csv', index = False)